# Advanced Machine Learning - Practicum 04 - Object Recognition

**Topics covered**: Apply two classifers, logistic regression and random forest to identify subjects from facial images.

**Deliverables**:
- Complete the tasks as detailed in this document.
- You are allowed to use scikit-learn packages.

**Objectives**:  
The objective of image recognition is to identify and portray, as a unique gray level (or color), the features occurring in an image in terms of the object on the ground. Image recognition is perhaps the most important part of digital image analysis. This tutorial will help you get familiarize with image object recognition using classifier. After completing it,  you will know:
- How to read image files.
- How to apply classifiers, e.g. logistic regression, random forest, for object recognition from images.

---
Import needed packages

In [1]:
import numpy as np 
from PIL import Image                                      # read image
from sklearn.linear_model import LogisticRegression        # logistic regression classifier
from sklearn.ensemble import RandomForestClassifier        # random forest classifier

## 1. Dataset
[olivettifaces](https://scikit-learn.org/0.19/datasets/olivetti_faces.html) is an image dataset. There are 10 different images of each of 40 distinct subjects. The image is quantized to 256 grey levels and stored as unsigned 8-bit integers. Below is the image containing all 400 samples.<br>
<img src="olivettifaces.gif" width="300" height='151'>
**olivettifaces** could be loaded by using sklearn.dataset packages. In this tutorial, in order to get familiarize with the image format, you are required to implement a `load_images()` function. It is used for loading the image file *olivettifaces.gif* and split it into data sets for train and test. <br>
Each sample from *olivettifaces* is an image with 57 by 47 pixels. Therefore the images will be loaded into a 400 by 2679(57\*47) array. The number of samples is 400, and each sample has a pixels/feature vector with size 2679. <br>
For the learing purpose the data should be further split into: 
- `train_data`:        360 samples
- `train_target`:      subject id (0-39) for samples from train_data 
- `test_data`:   40 samples
- `test_target`: subject id for samples from validation_data 

*hint*: It'd be good that facial images for all subjects participate in training.  

In [2]:
# Load face image data
def load_images(filename):

    image = Image.open(filename) # load the image
    data_imgs = np.asarray(image) # convert image to numpy array
    image_trimmed = np.delete(data_imgs, [471, 472], 1) #trim the vertical white line of the image @ col 471 n 472
    
    final = np.zeros(2679).reshape((1, 2679)) # initialize final var for vstack ops
    train_data = np.zeros(2679).reshape((1, 2679)) # initialize train_data for vstack ops
    test_data = np.zeros(2679).reshape((1, 2679)) # initialize test_data for vstack ops

    for imgs_in_a_row in np.vsplit(image_trimmed, 20): # split into 20 rows
        faces_20 = np.hsplit(imgs_in_a_row, 20) #split into 20 columns
        for each_face in faces_20: 
            face = np.reshape(each_face, (1, 2679))
            final = np.vstack((final, face))
    final = final[1:] # remove the initial zeros array
    
    for i in np.vsplit(final, 40):  # split final into 10 rows per i train and test
        train_data = np.vstack((train_data, i[:9])) # for every 10, takes 9 as train data
        test_data = np.vstack((test_data, i[-1])) # # for every 10, takes 1 as test data
   
    train_data = train_data[1:] # remove the initial zeros array
    test_data = test_data[1:]   # remove the initial zeros array
    ## ---------------------------------
    train_target = np.zeros((360, ))  # create train_target labels
    counter = 0 
    for i in range(40):  
        for j in range(9):
            train_target[counter] = i 
            counter += 1
    ## ---------------------------------
    test_target = np.zeros((40, ))  # create test_target labels
    counter = 0
    for j in range(40):
        test_target[counter] = j 
        counter += 1

    return train_data, train_target, test_data, test_target

In [3]:
# Get the data sets
train_data, train_target, test_data, test_target = load_images("olivettifaces.gif")
print(train_data.shape, train_target.shape, test_data.shape, test_target.shape)

(360, 2679) (360,) (40, 2679) (40,)


## 2. Logistic Regression
We've learn Logistic Regresson in the previous part of this module. <br>
You're to use the `LogisticRegression` function to trian a model on `train_data` and test if on the `test_data`. Type your code in the cell below, print out the accuracy on `train_data` and `test_data`. <br>
See the page [ref](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to learn the use the learning algorithm. The key to success is the parater `solver` which gives options on different gradient descent algorithm.

In [4]:
# Your code goes here
# for s in ['lbfgs', 'sag', 'saga', 'newton-cg']:
clf = LogisticRegression(solver='newton-cg', random_state=0)
clf.fit(train_data, train_target)
print(f'Test Score: {clf.score(test_data, test_target)}')

Test Score: 1.0


## 3. Random Forest

Random forest is a supervised, ensemble learning method for classification or regression. For a classification problem, it constructs many decision trees at training time and  merges them together at testing time to get a more accurate and stable prediction. <br>

In this task, use the `RandomForestClassifier` function to train the model on `train_data` and test if on the `test_data`. Type your code in the cell below, print out the accuracy on `train_data` and `test_data`. <br>
See the page [ref](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to learn the use the learning algorithm. <br>

In [5]:
# Your code goes here
# Create the model with 100 trees

clf = RandomForestClassifier(max_depth=200, random_state=0)
clf.fit(train_data, train_target)
print(f'Test score: {clf.score(test_data, test_target)}')

Test score: 0.975


---
## 4. Answer questions

- 1. What is the logistic regression model for prediction/classification? How many parameters to be learnt? <br>
Ans: Logistic regression is a statistical model that in its basic form uses a logistic function to model the probability of a certain class or event   existing in a binary sense, like 1 or 0, in our context. As above, each sample has a pixels/feature vector with size 2679, which corresponds to 2679 parameters to be learnt. 

    
- 2. How does the Random Forest learning algorithm used for real-valued features? <br>
Ans: Random forest onsists of a large number of individual decision trees that operate as an ensemble. Each individual tree in the random forest spits out a class prediction and the class with the most votes becomes our model’s prediction. Each tree in a random forest can pick only from a random subset of real-valued features. This forces even more variation amongst the trees in the model and ultimately results in lower correlation across trees and more diversification.